In [1]:
import pandas as pd
import numpy as np

In [4]:
car_sales = pd.read_csv('car-sales-extended-missing-data.csv')

FileNotFoundError: [Errno 2] File car-sales-extended-missing-data.csv does not exist: 'car-sales-extended-missing-data.csv'

In [ ]:
car_sales.head()

In [ ]:
car_sales.info()

In [ ]:
car_sales.isna().sum()

In [ ]:
# dropping the price with nan
car_sales.dropna(subset=['Price'],inplace=True)

In [ ]:
x = car_sales.drop('Price',axis=1)
y = car_sales.Price

In [ ]:
# clearing the nan out of the dataset using the sklearn method
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer

# columns in the data set to impute 
string_cols = ['Make','Colour']
number_cols = ['Odometer (KM)']
cat_cols = ['Doors']

# imputer setting 
string_imputer = SimpleImputer(strategy='constant',fill_value='missing')
number_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy = 'constant',fill_value=4)

# imputing 
imputer = ColumnTransformer([
    ('string imputer', string_imputer , string_cols),
    ('number imputer', number_imputer , number_cols),
    ('catogery imputer', cat_imputer , cat_cols )
],remainder = 'passthrough')

# transforming
transformed_x = imputer.fit_transform(x)

In [ ]:
new_x = pd.DataFrame(transformed_x,
                    columns= x.columns)

In [ ]:
new_x

In [ ]:
# converting to numbers 
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

# cols to trnsfrm
cols = ['Make','Colour','Doors']

# encoder setting 
transformer = ColumnTransformer([('applyingtheencodertotheneeded',
                                  encoder , 
                                  cols)],remainder='passthrough')

# we are now going to apply the transforming
numx = transformer.fit_transform(new_x)

# viewing the data 
its now clean from all the nan and stuff

In [ ]:
numx

In [ ]:
# creating the train test split 
from sklearn.model_selection import train_test_split

xtrain , xtest , ytrain , ytest = train_test_split(numx,y,test_size=0.2)

In [ ]:
# set a random seed for the prediction 
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

In [ ]:
model.fit(xtrain,ytrain)

# predicting with the model

In [ ]:
model.fit(xtrain,ytrain)
prediction = model.predict(xtest)
model.score(xtest,ytest)

In [ ]:
prediction[:10],ytest[:10]

In [5]:
# import matplotlib.pyplot as plt
# %matplotlib inline

In [6]:
# fig ,(ax,ax1) = plt.subplots(nrows=1,
#                             ncols=2,
#                             figsize=(10,10))

# ax.plot((prediction));
# ax1.plot(ytest );

# evaluating the models performance with some methods 
* 1. the simple R^2 method or the simple score method
* 2. the cross-validation method
* 3. the area under the ROC curve method
* 4. the confusion matrix method 

### 1.
**the simple score method**
like in the linear-regression , calculates the R^2 which ultimately turn out to be the score method .
> Return the coefficient of determination R^2 of the prediction.
>> the syntax : model.score(x_test_data , y_test_data)

In [7]:
model.score(xtest,ytest)

NameError: name 'model' is not defined

In [ ]:
prediction.T[:1220] , ytest

In [ ]:
prediction  = model.predict(xtest)
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(ytest.T)
plt.plot(prediction.T[:1087])

### 2.
**the scoring parameter or the cross_validation_score** uses another method where the test data is broken down into *cv* pieces
and then is used to calaculate in some other way . You know it ( how its done)
> determines the accureacy of the prediction by cross-validation 
>> syntax : cross_val_score(model,
X : array-like
    The data to fit.
, y:The target variable to try to predict in the case of
    supervised learning.
    ,cv, njobs_numofprocessors , nestimators )

In [ ]:
from numpy.random import seed
from sklearn.model_selection import cross_val_score
seed(42)
cross_val_score(model,numx,y,cv=5)
# here since the training data is numx and since its a supervised learining model ,we give in numx and y

### 3.
**the area under the ROC curve** is another method where a ROC or AUC curve is created and is compared 
> there is a use of false positive,true-positive , false-nve ,true-nve in this method
* false positive = model predicts 1 when truth is 0
* false neg = model pred0 when truth is 1
* true post = model pred1 when 1 
* true neg = model pred 0 when 1

In [ ]:
from sklearn.datasets import load_diabetes

In [ ]:
sugar = load_diabetes()

In [ ]:
sugar

In [ ]:
diabetes = pd.DataFrame(sugar['data'],
                       columns=sugar['feature_names'])
diabetes['target'] = sugar['target']

In [ ]:
diabetes.head()

In [ ]:
x = diabetes.drop('target',axis=1)
y = diabetes.target

In [ ]:
# we now can apply the ml algorithm to the data 
# here its going to be a regression model ( since it has got multiple values )
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error , mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

np.random.seed(42)

model = RandomForestRegressor()

xtrain ,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2)

model.fit(xtrain,ytrain)

In [ ]:
np.random.seed(42)
prediction = model.predict(xtest)

In [ ]:
mean_absolute_error(prediction,ytest) #literally means that each may have a var of ~44.7 difference from the mean of the target

In [ ]:
# amplitude of the error just the amplitude 
mean_squared_error(ytest,prediction) 

In [ ]:
# the score for the model using some of the metrics that we've used so far 
r2_score(ytest,prediction)*100 # performance of the model is just this 

In [ ]:
# accuracy score and the scoring parameter 
from sklearn.model_selection import cross_val_score as cvs

In [ ]:
# validating the model with simple accuracy method using cross validation 
cvs(model,x,y,cv=5,scoring=None)

# now ,hyperparameter tuning 
for both classification and regression models on the datasets from sklearn

In [ ]:
# just to define the classifier and regressor models efficiency 
from sklearn.metrics import precision_score,accuracy_score,f1_score,recall_score, r2_score,mean_absolute_error,mean_squared_error

# the overall quality check for classifiers 
def cmodel_eval(ytest,prediction):
    '''
    returns the models performance metrics 
    '''
    print({'accuracy score ':accuracy_score(ytest,prediction),
           'precision score':precision_score(ytest,prediction),
           'f1 score ':f1_score(ytest,prediction),
           'recall score':recall_score(ytest,prediction)})
           
def rmodel_eval(ytrue,prediction):
    print({'r squared':r2_score(ytrue,prediction),
          'MAE':mean_absolute_error(ytrue,prediction),
          'MSE':mean_squared_error(ytrue,prediction)})

In [ ]:
# classification problem
import pandas as pd
import numpy as np 

np.random.seed(23)

# the data frame creation 
data = pd.read_csv('heart-disease.csv')
data.isna().sum() 
data = data.sample(frac=1)

# create the x and y data 
x = data.drop('target',axis=1)
y = data.target

# creating a manual indexing of the train , validation split to ultimately make the test split 
train_indexing = round(0.7*len(x))
validation_indexing = round(train_indexing + 0.15*len(x))

# creating the train ,test and the validation split 
xtrain , ytrain = x[:train_indexing] , y[:train_indexing]
xvalid , yvalid = x[train_indexing:validation_indexing] , y[train_indexing:validation_indexing]
xtest , ytest = x[validation_indexing:] , y[validation_indexing:]

# we now make the model 
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

# fitting the model to the data which we have 
model.fit(xtrain,ytrain)

# now the model is going to be predicting on the validation set or the eval set 
prediction = model.predict(xvalid)

# now we are gonna evaluate the models performance on the base that it is a classification model 
cmodel_eval(yvalid,prediction)

In [ ]:
model.get_params()

In [ ]:
# now that we got the predictions evaluated , lets try imporve the performance of our model using the randomisezcv method
from sklearn.model_selection  import RandomizedSearchCV

# the params that we could try alter to get a good imporvement into a grid 
grid = {'max_depth':(12,5,10,15,20),
       'max_features':['auto','sqrt'],
       'max_leaf_nodes':(5,10,20,30,40),
       'n_estimators':(10,100,3000,2000,5000),
       'min_samples_leaf':(5,6,7,9),
       'min_samples_split':(2,4,6,8)}# which essentially means all the processors

# tuning the model onto the param distributions
randomised_model = RandomizedSearchCV(estimator = model,
                                     param_distributions=grid,
                                     n_iter=10,
                                     cv=5
                                     ,verbose=2,n_jobs=4)

# fitting the model again
randomised_model.fit(xtrain,ytrain)

In [ ]:
randomised_model.best_params_

In [ ]:
final_prediction = randomised_model.predict(xvalid)

In [ ]:
final_prediction

In [ ]:
cmodel_eval(yvalid,final_prediction)

# now gonna put it all together using the pipeline module from sklearn

In [ ]:
data.Doors

In [ ]:
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder 
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd 

# we go\nna do it all the way using the pipeline module 
# but first , we gonna need the data
data = pd.read_csv('car-sales-extended-missing-data.csv')

# we have to predict the number of doors of the car using its other features
data.dropna(subset=['Doors'],inplace=True)
x = data.drop('Doors',axis=1)
y = data.Doors

# now we have to use the imputer to impute the NaN rows inside the dataframe
cat_cols = ['Make','Colour']
cat_transformer = Pipeline(steps=[
    ('imputation on cat cols', SimpleImputer(strategy='constant',fill_value='missing'))
])



num_cols = ['Odometer (KM)','Price']
num_transformer = Pipeline(steps = [
    ('imputation', SimpleImputer(strategy='mean')),
])

# dont forget to transform the column 
transformer = ColumnTransformer([
    ('the cat pline', num_transformer , cat_cols),
    ('the num pline',num_transformer , num_cols)
])

# we now have to fit the transformation 
transformed_x = transformer.fit_transform(x)

# now that the data is imputed and encoded , we are going to  create the train and test split
xtrain,xtest,ytrain,ytest = train_test_split(transformed_x,y,test_size=0.2)

# instantiating the model 
model = Pipeline(steps=[
    ('preprocessing',transformer),
    ('model type',RandomForestRegressor())
])

# time to fit the model on the data
model.fit(xtrain,ytrain)

# combining all together 

In [8]:
data =pd.read_csv('/Users/hp/Desktop/ml/data/car-sales-extended-missing-data.csv')

In [10]:
#  we have null values in most part of our data 
# target variable is going to be the doors 
data.dropna(subset=['Doors'],inplace=True)

In [11]:
data.isna().sum() # now the target variable is cleaned

Make             47
Colour           46
Odometer (KM)    49
Doors             0
Price            47
dtype: int64

In [13]:
data.dtypes

Make              object
Colour            object
Odometer (KM)    float64
Doors            float64
Price            float64
dtype: object

In [12]:
import numpy as np

In [24]:
# cleaning the data using the pipeline 
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

np.random.seed(42)

# preprocessing steps
# seting the feature variables 
str_cols = ['Make',"Colour"]
str_transformer = Pipeline(steps= [
    ('string imputing',SimpleImputer(strategy = 'constant',fill_value='missing')),
    ('string encoding',OneHotEncoder(handle_unknown='ignore'))
])

num_cols =['Odometer (KM)','Price']
num_transformer = Pipeline(steps=[
    ('numeric imputer',SimpleImputer(strategy='mean'))
])

# linking the above pipelines to the columnTransformer
transformer = ColumnTransformer([
    ('str transformer', str_transformer , str_cols ),
    ('num transformer', num_transformer , num_cols )
],remainder = 'passthrough')

# linking the transformer( s ) to a model  > pre
model = Pipeline(steps=[
     ('preprocessor',transformer ),
     ('rfr_model',RandomForestRegressor())
])


# splitting the data 
x = data.drop('Doors',axis=1)
y = data.Doors

xtrain , xtest , ytrain, ytest = train_test_split(x,y,test_size=0.2)

# model fitting 
model.fit(xtrain,ytrain)
model.score(xtest)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [25]:
xtrain.isna().sum() , ytrain.isna().sum()

(Make             41
 Colour           33
 Odometer (KM)    41
 Price            38
 dtype: int64,
 0)

In [26]:
xtrain ,ytrain

(       Make Colour  Odometer (KM)    Price
 986   Honda  White        71934.0  26882.0
 307  Nissan   Blue       174825.0   4863.0
 578  Toyota   Blue       215955.0   7994.0
 290  Toyota  Black       198564.0   7708.0
 115  Toyota  Black        20714.0  30407.0
 ..      ...    ...            ...      ...
 112  Nissan  White        72122.0  22943.0
 284   Honda    Red       238609.0   5052.0
 909     BMW  White       232696.0  26940.0
 461     BMW  White       143651.0  22597.0
 108  Nissan   Blue       129769.0  15617.0
 
 [760 rows x 4 columns],
 986    4.0
 307    3.0
 578    4.0
 290    4.0
 115    4.0
       ... 
 112    4.0
 284    4.0
 909    5.0
 461    5.0
 108    3.0
 Name: Doors, Length: 760, dtype: float64)